# `Employee_Names_and_IDs.sql`

## Proposition
Retrieve the first name, last name, and ID of employees from the AdventureWorks2019 database.

## Tables

- `Person.Person` (aliased as `p`)
- `HumanResources.Employee` (aliased as `h`)
- `Person.BusinessEntity` (aliased as `b`)

## Columns

- `p.FirstName`
- `p.LastName`

In [ ]:
USE AdventureWorks2019

SELECT p.FirstName, p.LastName
FROM Person.Person AS p
         INNER JOIN HumanResources.Employee AS h
                    ON p.BusinessEntityID = h.BusinessEntityID
         INNER JOIN Person.BusinessEntity AS b
                    ON b.BusinessEntityID = h.BusinessEntityID

___
___
___

# `Sales_Order_Product_Customer.sql`

## Proposition
Retrieve details about each sales order, including the order date, the customer’s full name, the product name, the unit price, the quantity ordered, and the total sales amount. This will help analyze customer purchases and product sales performance.

## Tables

- `SalesOrderHeader`
- `SalesOrderDetail`
- `Product`
- `Customer`
- `Person`

## Columns

- `SalesOrderHeader.OrderDate`
- `SalesOrderHeader.CustomerID`
- `SalesOrderDetail.ProductID`
- `SalesOrderDetail.OrderQty`
- `SalesOrderDetail.UnitPrice`
- `SalesOrderDetail.LineTotal`
- `Product.Name`
- `Customer.CustomerID`
- `Person.FirstName`
- `Person.LastName`

In [ ]:
USE AdventureWorks2019

SELECT soh.OrderDate,
       p.FirstName + ' ' + p.LastName AS CustomerFullName,
       prd.Name                       AS ProductName,
       sod.OrderQty,
       sod.UnitPrice,
       sod.OrderQty * sod.UnitPrice   AS TotalAmount
FROM Sales.SalesOrderHeader AS soh
         JOIN
     Sales.SalesOrderDetail AS sod
     ON soh.SalesOrderID = sod.SalesOrderID
         JOIN
     Production.Product AS prd
     ON sod.ProductID = prd.ProductID
         JOIN
     Sales.Customer AS c
     ON soh.CustomerID = c.CustomerID
         JOIN
     Person.Person AS p
     ON c.PersonID = p.BusinessEntityID
ORDER BY soh.OrderDate DESC;

___
___
___

# `Customer_Order_Salesperson_Details.sql`

## Proposition

Identify all customers who placed an order along with details of the product they ordered, the sales person who handled the order, and the total amount of the order.

## Tables

- `Sales.Customer`: to get customer information.
- `Sales.SalesOrderHeader`: to get the order information.
- `Sales.SalesOrderDetail`: to get details of the order, including the products.
- `Sales.SalesPerson`: to get the salesperson handling the order.
- `Production.Product`: to get details of the products being ordered.

## Columns

- `Sales.Customer`: `CustomerID`, `PersonID`
- `Sales.SalesOrderHeader`: `SalesOrderID`, `CustomerID`, `SalesPersonID`, `TotalDue`
- `Sales.SalesOrderDetail`: `SalesOrderID`, `ProductID`, `OrderQty`
- `Sales.SalesPerson`: `SalesPersonID`, `FirstName`, `LastName`
- `Production.Product`: `ProductID`, `Name` (Product name)

In [ ]:
USE AdventureWorks2019

SELECT c.CustomerID,
       p.FirstName + ' ' + p.LastName AS SalesPersonName,
       soh.SalesOrderID,
       pr.Name                        AS ProductName,
       sod.OrderQty,
       soh.TotalDue
FROM Sales.Customer c
         JOIN
     Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
         JOIN
     Sales.SalesOrderDetail sod ON soh.SalesOrderID = sod.SalesOrderID
         JOIN
     Production.Product pr ON sod.ProductID = pr.ProductID
         JOIN
     Sales.SalesPerson sp ON soh.SalesPersonID = sp.BusinessEntityID
         JOIN
     Person.Person p ON sp.BusinessEntityID = p.BusinessEntityID
ORDER BY soh.SalesOrderID;

___
___
___

# `Total_Sales_Per_Category.sql`

## Proposition
Find the total sales amount for each product category, along with the number of orders placed for products in each category.

## Tables
- `Sales.SalesOrderDetail`: to get product and order details.
- `Production.Product`: to get product information.
- `Production.ProductSubcategory`: to categorize products into subcategories.
- `Production.ProductCategory`: to group subcategories into categories.

## Columns
- `Sales.SalesOrderDetail`: `ProductID`, `LineTotal`
- `Production.Product`: `ProductID`, `ProductSubcategoryID`
- `Production.ProductSubcategory`: `ProductSubcategoryID`, `ProductCategoryID`
- `Production.ProductCategory`: `ProductCategoryID`, `Name` (Category name)

In [ ]:
USE AdventureWorks2019

SELECT pc.Name                 AS CategoryName,
       COUNT(soh.SalesOrderID) AS NumberOfOrders,
       SUM(sod.LineTotal)      AS TotalSalesAmount
FROM Sales.SalesOrderDetail sod
         JOIN
     Production.Product p ON sod.ProductID = p.ProductID
         JOIN
     Production.ProductSubcategory ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
         JOIN
     Production.ProductCategory pc ON ps.ProductCategoryID = pc.ProductCategoryID
         JOIN
     Sales.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
GROUP BY pc.Name
ORDER BY TotalSalesAmount DESC;

___
___
___

# `Employee_Department_Manager.sql`

## Proposition
Retrieve the list of employees along with their job title, department, and their respective manager’s name (if applicable).

## Tables
- `HumanResources.Employee`: to get employee information.
- `HumanResources.EmployeeDepartmentHistory`: to get the department information.
- `HumanResources.Department`: to get department details.
- `HumanResources.Shift`: to get the shift information (optional for added details).
- `Person.Person`: to get the employee's and manager's names.
- `HumanResources.Employee` (for self-join to get manager information).

## Columns
- `HumanResources.Employee`: `BusinessEntityID`, `JobTitle`, `ManagerID`
- `HumanResources.EmployeeDepartmentHistory`: `BusinessEntityID`, `DepartmentID`
- `HumanResources.Department`: `DepartmentID`, `Name` (Department Name)
- `Person.Person`: `BusinessEntityID`, `FirstName`, `LastName`

In [ ]:
USE AdventureWorks2019

SELECT e.BusinessEntityID               AS EmployeeID,
       p.FirstName + ' ' + p.LastName   AS EmployeeName,
       e.JobTitle,
       d.Name                           AS DepartmentName,
       p2.FirstName + ' ' + p2.LastName AS ManagerName
FROM HumanResources.Employee e
         JOIN
     Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
         JOIN
     HumanResources.EmployeeDepartmentHistory edh ON e.BusinessEntityID = edh.BusinessEntityID
         JOIN
     HumanResources.Department d ON edh.DepartmentID = d.DepartmentID
         LEFT JOIN
     HumanResources.Employee e2 ON e.BusinessEntityID = e2.BusinessEntityID
         LEFT JOIN
     Person.Person p2 ON e2.BusinessEntityID = p2.BusinessEntityID
WHERE edh.EndDate IS NULL -- Fetch only current department assignments
ORDER BY EmployeeName;